# Ciencia de datos de clasificación

Importar librerías

In [1]:
import pandas as pd
import numpy as np

Importar datos limpios

In [2]:
df=pd.read_csv("listings_limpio.csv")

Visualizar data frame

In [3]:
df.head(5)

,id,host_response_rate,host_acceptance_rate,neighbourhood_cleansed,latitude,longitude,accommodates,bathrooms,bedrooms,beds,...,Safe,Refrigerator,Smoke_alarm_home_safety,Essentials,Services,property_Entire_Place,property_Hotel_Room,property_Other,property_Private_Room,property_Shared_Room
0,30320,100,100,Sol,40.41476,-3.70418,2,1.0,1.0,2.0,...,0,0,0,0,0,True,False,False,False,False
1,40916,100,100,Universidad,40.42247,-3.70577,2,1.0,1.0,3.0,...,0,0,0,0,0,True,False,False,False,False
2,62423,100,99,Justicia,40.41884,-3.69655,4,1.5,1.0,2.0,...,0,1,1,1,0,False,False,False,True,False
3,70073,100,100,Universidad,40.42252,-3.70250,2,1.0,1.0,1.0,...,0,1,0,1,1,True,False,False,False,False
4,70310,100,100,Universidad,40.42252,-3.70250,2,1.0,1.0,2.0,...,0,1,0,1,1,True,False,False,False,False


Visualizar los ratings disponibles

In [4]:
df_ratings = df.filter(like="review")
df_ratings.head(5)

,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,173,4.63,4.71,4.88,4.82,4.78,4.90,4.69,0.93
1,53,4.68,4.71,4.90,4.87,4.81,4.88,4.59,0.29
2,249,4.65,4.78,4.46,4.80,4.86,4.97,4.60,2.78
3,36,4.23,4.49,4.42,4.43,4.40,4.60,4.09,0.21
4,31,4.37,4.65,4.55,4.80,4.71,4.87,4.10,0.18


In [5]:
ratings=df_ratings['review_scores_rating']
np.percentile(ratings,65)

4.8